In [5]:
# install Amumo

# need ipykernel > 6
# ! pip install ipykernel==6.23.1
# ! pip install git+https://github.com/ginihumer/Amumo.git[clip,cloob,diffusion-db]
# for local installation
# ! pip install -e ..[clip,cloob,diffusion-db]

  Cloning https://github.com/facebookresearch/ImageBind to c:\users\christina\appdata\local\temp\pip-req-build-yarpu46c
  Resolved https://github.com/facebookresearch/ImageBind to commit c6a47d6dc2b53eced51d398c181d57049ca59286
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pytorchvideo-0.1.5-py3-none-any.whl
  Using cached torch-1.13.0-cp39-cp39-win_amd64.whl (167.2 MB)
  Using cached torchvision-0.14.0-cp39-cp39-win_amd64.whl (1.1 MB)
  Using cached torchaudio-0.13.0-cp39-cp39-win_amd64.whl (2.1 MB)
  Using cached timm-0.6.7-py3-none-any.whl (509 kB)
  Using cached einops-0.7.0-py3-none-any.whl.metadata (13 kB)
  Using cached fvcore-0.1.5.post20221221-py3-none-any.whl
  Using cached eva_decord-0.6.1-py3-none-win_amd64.whl (25.5 MB)
  Using cached iopath-0.1.10-py3-none-any.whl
  Using cached matplotlib-3.8.2-cp39-cp39-win_amd64.whl.metadata (5.9 kB)
  Using cached types_regex-2023.12.25.20240106-py3-none-any.whl.me

  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/ImageBind 'C:\Users\Christina\AppData\Local\Temp\pip-req-build-yarpu46c'
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
amumo 0.1.38 requires torch==2.2.0, but you have torch 1.13.0 which is incompatible.


In [4]:
! pip freeze

absl-py==1.1.0
aiohttp==3.9.3
aiosignal==1.3.1
-e git+https://github.com/ginihumer/Amumo.git@0959bbd7c6765ed67b3adb94bf05d319e01f3958#egg=Amumo
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
asttokens==2.0.5
astunparse==1.6.3
async-timeout==4.0.3
attrs==23.2.0
backcall @ file:///home/conda/feedstock_root/build_artifacts/backcall_1592338393461/work
beautifulsoup4==4.11.1
bleach==5.0.1
bqplot==0.12.33
cachetools==5.2.0
certifi==2023.11.17
cffi==1.15.1
charset-normalizer==3.3.2
colorama @ file:///home/conda/feedstock_root/build_artifacts/colorama_1666700638685/work
comm @ file:///home/conda/feedstock_root/build_artifacts/comm_1704278392174/work
datasets==2.14.6
debugpy @ file:///C:/b/abs_c0y1fjipt2/croot/debugpy_1690906864587/work
decorator==5.1.1
defusedxml==0.7.1
dill==0.3.7
executing @ file:///home/conda/feedstock_root/build_artifacts/executing_1698579936712/work
filelock==3.13.1
frozenlist==1.4.1
fsspec==2023.10.0
gast==0.4.0
google-auth==2.9.0
google-auth-oauthlib==0.4.6
google-pas

In [1]:
# import 
from amumo import model as am_model
from amumo import data as am_data
from amumo import widgets as am_widgets

C:\Users\Christina\AppData\Roaming\Python\Python39\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\Christina\AppData\Roaming\Python\Python39\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\Christina\AppData\Roaming\Python\Python39\site-packages\uma

# Basic usage

In [2]:
# load dataset
dataset = am_data.DiffusionDB_Dataset(path="2m_first_1k", batch_size=100) # data helper for the diffusionDB dataset; for the interactive prototype, we only use a random subset of 100 samples
all_images, all_prompts = dataset.get_data()
cache_name = 'diffusiondb_random_100' # path used to cache the results

In [3]:
# This widget consists of a similarity heatmap that shows the similarities between all latent embeddings, a 2-dimensional projection of the embeddings, and a hover view of the currently hovered instance.
# It also allows user input for choosing the model, an option to manually close the modality gap, an option to cluster the matrix rows by their similarity, and settings for the 2-dimensional projection. 
# To learn more about the proper use for closing the modality gap, see the interactive article: https://jku-vds-lab.at/amumo.
# You can specify a list of models to compare; there is a predefined set of models, but you can also pass your own model class.
# The widget takes a list of images and a list of prompts of the same size as input. Images and texts with the same index are considered pairs.
am_widgets.CLIPExplorerWidget(cache_name, all_data={"image": all_images, "text": all_prompts}, models=['CLIP', am_model.CyCLIPModel(), am_model.CLOOB_LAION400M_Model()]) 

c:\Users\Christina\anaconda3\envs\amumo_test\lib\site-packages\torchaudio\backend\utils.py:62: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")
c:\Users\Christina\anaconda3\envs\amumo_test\lib\site-packages\torchvision\transforms\_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
c:\Users\Christina\anaconda3\envs\amumo_test\lib\site-packages\torchvision\transforms\_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


found cached embeddings for diffusiondb_random_100_CLIP_RN50 image
found cached embeddings for diffusiondb_random_100_CLIP_RN50 text


CLIPExplorerWidget(children=(VBox(children=(HBox(children=(Dropdown(description='Model: ', options=('CLIP', 'C…

batch 1 of 1
batch 1 of 1


In [ ]:
# This widget shows similarity heatmaps for all models in the list for a better comparison between models.
am_widgets.CLIPComparerWidget(cache_name, all_images, all_prompts, models=['CLIP', am_model.CyCLIPModel(), am_model.CLOOB_LAION400M_Model()]) 

True [False, False, False] {'CLIP': <amumo.model.CLIPModel object at 0x000001A3B6609370>, 'CyCLIP': <amumo.model.CyCLIPModel object at 0x000001A3875B9DF0>, 'CLOOB_LAION400M': <amumo.model.CLOOB_LAION400M_Model object at 0x000001A3877EEF10>}
found cached embeddings for diffusiondb_random_100_CLIP_RN50


c:\users\christina\repositories\icg\researchstay\amumo\amumo\widgets.py:940: FutureWarning:

The input object of type 'PngImageFile' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'PngImageFile', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.

c:\users\christina\repositories\icg\researchstay\amumo\amumo\widgets.py:940: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



found cached embeddings for diffusiondb_random_100_CyCLIP_RN50
found cached embeddings for diffusiondb_random_100_CLOOB-LAION400M_ViT-B-16


CLIPComparerWidget(children=(HoverWidget(children=(VBox(children=(HTML(value='', layout=Layout(width='300px'))…

# Use a Custom Model

In [ ]:
import clip
import torch
import numpy as np
# You can use the CLIPModelInterface to create your own model class wrapper.
class CustomModel(am_model.CLIPModelInterface):
    available_models = clip.available_models()
    model_name = 'MyCLIP'

    def __init__(self, name='RN50', device='cpu') -> None:
        super().__init__(name, device)
        self.model, self.preprocess = clip.load(name, device=device)
        self.model.eval()
        self.logit_scale = self.model.logit_scale

    def encode_image(self, images):
        images = [self.preprocess(i) for i in images]
        image_input = torch.tensor(np.stack(images)).to(self.device)
        return self.model.encode_image(image_input).float().cpu()

    def encode_text(self, texts):
        text_tokens = clip.tokenize(texts, truncate = True).to(self.device)
        return self.model.encode_text(text_tokens).float().cpu()

In [ ]:
# The widget takes any instance of type CLIPModelInterface.
am_widgets.CLIPExplorerWidget(cache_name, all_data={"image": all_images, "text": all_prompts}, models=[CustomModel()])